In [1]:
import numpy as np
import pandas as pd
import shutil
import random
import os

# Import OpenImagesV7 for background images

https://storage.googleapis.com/openimages/web/download_v7.html#download-manually

In [ ]:
class_labels_url = "https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions-boxable.csv"
images_ID_url = "https://storage.googleapis.com/openimages/v5/train-annotations-human-imagelabels-boxable.csv"

class_labels = pd.read_csv(class_labels_url)
images_ID = pd.read_csv(images_ID_url)

In [ ]:
class_labels.head()

In [ ]:
images_ID.head()

In [ ]:
Human_labels = class_labels[class_labels['DisplayName'].str.contains('Human')]["LabelName"].values
Plant_label = class_labels[class_labels['DisplayName'] == "Plant"]["LabelName"].values[0]
Human_labels, Plant_label

In [ ]:
image_id_with_humans = set(images_ID[(images_ID["LabelName"].isin(Human_labels)) & (images_ID["Confidence"] == 1)]["ImageID"].values)
image_id_with_plants = set(images_ID[(images_ID["LabelName"] == Plant_label) & (images_ID["Confidence"] == 1)]["ImageID"].values)

image_id_with_plants_or_human_labeled = image_id_with_humans | image_id_with_plants
image_id_with_plants_or_human_labeled = list(image_id_with_plants_or_human_labeled)

# Remove the images with plants or humans labeled in it.
df_image_id_without_plants_or_human_labeled = images_ID[~images_ID["ImageID"].isin(image_id_with_plants_or_human_labeled)]
len(df_image_id_without_plants_or_human_labeled[df_image_id_without_plants_or_human_labeled["ImageID"].isin(image_id_with_plants_or_human_labeled)]) == 0 

In [ ]:
list_of_image = list(set(list(df_image_id_without_plants_or_human_labeled["ImageID"].values)))

In [ ]:
with open("image_id_without_plants_or_human_labeled.txt", "w") as f:
    for image_id in list_of_image[:10000]:
        f.write(f"train/{image_id}\n")

In [ ]:
with open("image_id_without_plants_or_human_labeled.txt", "r") as f:
    text = f.readlines()
    for line in text[:10]:
        print(line)

In [ ]:
! wget https://raw.githubusercontent.com/openimages/dataset/master/downloader.py

In [ ]:
! python downloader.py /kaggle/working/image_id_without_plants_or_human_labeled.txt --download_folder=/kaggle/working/subset_OpenImagesV7 --num_processes=5

In [ ]:
shutil.make_archive('subset_OpenImagesV7', 'zip', '/kaggle/working/subset_OpenImagesV7')